In [ ]:
!ls

'1단계_코드구현 및작동'
 all_examples_train.txt
 all_index_sentences_test.txt
 all_index_sentences_train.txt
 all_index_senterces_validation.txt
 all_word_pos_sentences_all.txt
 all_word_pos_sentences_test.txt
 all_word_pos_sentences_train.txt
 all_word_pos_sentences_validation.txt
 all_word_pos_sentenes_train.txt
 autoencoder_denoising.h5
'Benchmark_FixRes.ipynb의 사본'
'CNN2018 - 5 - [Class Exercise] CNN Case Studies.ipynb의 사본'
 Denoising_001.ipynb
'Denoising_001.ipynb의 사본'
'Denoising_001.ipynb의 사본 (1)'
'Denoising_001.ipynb의 사본 (2)'
'Denoising_001.ipynb의 사본 (3)'
'Denoising_001.ipynb의 사본 (4)'
'Denoising_001.ipynb의 사본 (5)'
 Denoising_V3.07_제출용.ipynb
 file_names_all.txt
 file_names_test.txt
 file_names_train.txt
 file_names_validation.txt
 fix2.ipynb
 FixRes_001.ipynb
'FixRes_001.ipynb의 사본'
 kagglecatsanddogs_3367a.zip
 kimsunju.ipynb
'MSR-LA - 3467.docx'
'MURA 001.ipynb'
 MURA_densenet_001
'MURA_densenet_001의 사본'
'MURA_densenet_001의 사본

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd gdrive/My Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [3]:
# 불러오는 부분
 
import os 
import time 
import tensorflow as tf 
from tensorflow. keras . layers import Embedding 
from tensorflow. keras import layers 
from tensorflow import keras 
import numpy as np 
d_embed = 100
window_size = 5
Threshold_freq =1

In [4]:
fp = open("file_names_all.txt", "w", encoding="utf-8")
cur_dir_path = './WSJ'
dir_list = os.listdir(cur_dir_path)
list_all_file_paths = []
for a_dir in sorted(dir_list):
    path = os.path.join(cur_dir_path, a_dir)
    subdir_list = os.listdir(path)
    for a_file in sorted(subdir_list):
        if not(a_file[-1]=='S' and a_file[-2]=='0'):
            continue
        path2 = os.path.join(path, a_file)
        fp.write(path2+'＼n')
        list_all_file_paths.append(path2)
fp.close()
 
# divide the whole corpus to 3 parts! training. validation. test
fp_tr = open("./file_names_train.txt", "w", encoding="utf-8")
fp_va = open("./file_names_validation.txt", "w", encoding="utf-8")
fp_te = open("./file_names_test.txt", "w", encoding="utf-8")
 
total_num_files = len(list_all_file_paths)
num_files_train = int(total_num_files * 0.8)
num_files_validation = int(total_num_files * 0.1)
num_files_test = total_num_files - (num_files_train + num_files_validation)
 
for i in range(num_files_train):
    fp_tr.write(list_all_file_paths[i] + '\n')
fp_tr.close()
 
for i in range(num_files_validation):
    fp_va.write(list_all_file_paths[num_files_train + i] + '\n')
fp_va.close()
 
for i in range(num_files_test):
    fp_te.write(list_all_file_paths[num_files_train + num_files_validation + i] + '\n')
fp_te.close()

In [5]:
def build_word_pos_sentences(path_file_names, path_result_file):
    fp_n = open(path_file_names, "r", encoding="utf-8")
    fp_s = open(path_result_file, "w", encoding="utf-8")
 
    while True:
        filename = fp_n.readline()
 
        if len(filename) == 0:
            break     # enf of file reached.
 
        filename = filename[:-1]
        sentences = read_a_file_and_form_sentences(filename)
        fp_s.write('<<' + filename + '>> Num sentences:' + str(len(sentences)) + '\n')
 
        # store word/pos in file all word pos seaence ????.txt'.
        for sentence in sentences:
            s_line = '' # initialize with emotv string.
            for w_p in sentence:
                s_line = s_line + '' + w_p
            fp_s.write(s_line + '\n')
    fp_n.close()
    fp_s.close()
 
build_word_pos_sentences("./file_names_all.txt", "./all_word_pos_sentences_all.txt")
 
 
build_word_pos_sentences("./file_names_train.txt", "./all_word_pos_sentences_train.txt")
build_word_pos_sentences("./file_names_validation.txt", "./all_word_pos_sentences_validation.txt")
build_word_pos_sentences("./file_names_test.txt", "./all_word_pos_sentences_test.txt")

In [6]:
def read_a_file_and_from_sentences (file_path):
    fp = open(file_path, "r", encoding="utf-8")
 
    sentences_of_file = [] # this will have lists. where each list will have word/pos Dairs of a sentence.
 
    while True: # this is for a sentence
        sentence = []
        while True:
            line = fp.readline()
            leng = len(line)
            if leng == 0:
                # end of file has come. So finish this file. Terminate this function with returing the result
                # note that there is nothing to insert sentence into sentences of file.
                return sentences_of_file
            elif leng == 1 or (line[0] == '=' and line[1] == '=' and line[2] == '='):
                continue
            else:
                # first line of a sentence has come. Store its elements.
                line = line[:-1]
                line_splited = line.split()
                for ele in line_splited:
                    if ele != '[' and ele != ']':
                        sentence.append(ele)
                break
 
        while True:
            line = fp.readline()
            leng = len(line)
            if leng == 0: # end of file has come. so finish this file.
                break
            elif leng == 1 or (leng >= 4 and (liner[0] == '=' and line[1] == '=' and liner[2] == '=')):
                # the first garbage line after the last line of a sentence has come.
                break
            else:
                # next line of a sentence has come.
                line = line[:-1]
                line_splited = line.split()
 
                for ele in line_splited:
                    if not(ele == '[' or ele == ']'):
                        sentence.append(ele)
 
        sentences_of_file.append(sentence)
 
        if leng == 0:    # the end of file has come. So we need to terminate the function.
            break
    return sentences_of_file

In [7]:
def build_vocabulary_temp(corpus_path):
    fp = open(corpus_path, "r", encoding="utf-8")
    for line in fp.readlines():
        sentence = line.split()
        if sentence[0] == '<<':
            continue
        for word_pos_pair in sentence:
            w_p = word_pos_pair.split('/')
            word = w_p[0]
            pos = w_p[1]
            if word in Vocab_temporary:
                Vocab_temporay[word] += 1
            else:
                Vocab_temporay[word] = 1
    fp.close()
    return
 
# Create Vocab(a Vocabulary) of our system by taking words (in train part of the tagged corpus)
# of fred greater or equal to a given threshold.
 
Vocab_temporary = {}
build_vocabulary_temp("./all_word_pos_sentences_all.txt")
sorted_Vocab = sorted(Vocab_temporary.items(), key = lambda kv: kv[1], reverse=True) # this
Total_n_words = len(sorted_Vocab)
 
Vocab = {} # This will be our final vocab.
n_reserved = 2 # currently. 「PAD1 of index 0. [UNK] of index 1.
for i in range(Total_n_words):
    word = sorted_Vocab[i][0]
    freq = sorted_Vocab[i][1]
 
    if fred >= Threshold_freq:
        Vocab[word] = i + n_reserved # give index to word.
    else:
        # all the cemaining will be ones of fred less than threshold frea.
        break
 
Vocab['[PAD]'] = 0 # index of pad word
Vocab['[UNK]'] = 1 # index of unknown word
Vocab_size = len(Vocab) # include 「PAD1. 「UNK1
 
# Create our dictionary for part of speech, dic POS, from original paper of Penn-tree bank
dic_POS = {}
all_pos_list = ['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS', \
                'MD','NN','NNS','NNP','NNPS','PDT','POS', 'PRP', 'PRP$','RB', \
                'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', \
                'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '#', '$', '.', ',', \
                ':', '(', ')']
# give a unique index to each POS.
for i in range(43):
    dic_POS[all_pos_list[i]] = i

In [8]:
def build_index_sentences(path_word_pos_sentence_file, path_index_sentence_file):
    fp = open(path_word_pos_sentence_file, "r", encoding="utf-8")
    fp_w = open(path_index_sentence_file, "w", encoding="utf-8")
 
    for line in fp.readlines():
        sentence = line.split()
        if sentence[0] == '<<':
            continue
 
        line_widx = '' # line for word indices
        line_pidx = '' # line for DOS indices
 
        for word_pos_pair in sentence:
            w_p = word_pos_pair.split('/')
            nseg = len(w_p)
            if nseg > 2:
                word = ''
                for i in range(nseg-1):
                    word = word + w_p[i] + '/'
                word = word[:-1] # remove the last slash.
            else:
                word = w_p[0]
 
            Pos = w_p[-1] # the last segment
            if not(word in Vocab):
                widx = 1 # give index of 「UNK1 since it is missing in Vocab.
            else:
                widx = Vocab[word]
 
            if not(pos in dic_POS):
                pos_list = pos.split('|')
                pos = pos_list[-1]
                if not (pos in dic_POS):
                    print("exception occurs at dic POS look up. W D=", w_p, " pos=", pos)
                    time.sleep(100)
                else:
                    pidx = dic_POS[pos]
            else:
                pidx = dic_POS[pos]
 
            if len(line_widx) == 0:
                line_widx = line_widx + str(widx)
            else:
                line_widx = line_widx + '\t' + str(widx)
 
            if len(line_pidx) == 0:
                line_pidx = line_pidx + str(pidx)
            else:
                line_pidx = line_pidx + '\t' + str(pidx)
 
        fp_w.write(line_widx + '\n')
        fp_w.write(line_pidx + '\n')
        fp_w.write:('\n') # an empty line after each sentence
    fp_w.close()
    fp.close()
 
build_index_sentences("./all_word_pos_sentences_train.txt", "./all_index_sentences_train.txt")
build_index_sentences("./all_word_pos_sentences_validation.txt", "./all_index_sentences_validation.txt")
build_index_sentences("./all_word_pos_sentences_test.txt", "./all_index_sentences_test.txt")

In [9]:
def build_examples(path_index_file, path_examples_file, window_size):
    fp = open(path_index_file, "r", encoding="utf-8")
    fp_w = open(path_examples_file, "w", encoding="utf-8")
 
    fp_w.write(str(window_size)+" word indices and a pos label.\n")
          # insert first line as comment.
    num_pads = window_size // 2
    while True:
        # read two lines
        wline = fp.readline()
        if len(wline) == 0:
            break # end of file has come.
        pline = fp.readline()
        w_index = wline.split()
        p_index = pline.solit()
 
        # pad (index 0) are placed before the sentence
        pad_added_word_index = []
        for i in range(num_pads):
            pad_added_word_index.append('0')
        
        for widx in w_index:
            pad_added_word_index.append(widx)
        
        # pade are added after sentence
        for i in range(num_pads):
            pad_added_word_index.append('0')
        
        leng_sentence = len(w_index)
        for i in range(leng_sentence):
            outline = ''
            for j in range(window_size):
                outline += pad_added_word_index[i+j]
                outline += '\t'
            outline += p_index[i] + '\n'
            fp_w.write(outline)
 
        line = fp.readline()
        if len(line) != 1:
            print("logic error. length 1 line expected in file all index sentences.txt")
            time.sleep(100)
        
    fp.close()
    fp_w.close()
 
build_examples("./all_index_sentences_train.txt", "./all_examples_train.txt", window_size)
build_examples("./all_index_sentences_validation.txt", "./all_examples_validation.txt", window_size)
build_examples("./all_index_sentences_test.txt", "./all_examples_test.txt", window_size)

In [10]:
# Model design
embedding_layer = Embedding(Vocab_size, d_embed, embeddings_initializer='random_normal', 
                            input_length=window_size, trainable=True)
int_seq = keras.Input(shape=(window_size), dtype="int64")
emb_seq = embedding_layer(int_seq)
x = layers.Flatten()(emb_seq)
x = layers.Dense(640, activation="relu")(x)
x = layers.Dense(640, activation="relu")(x)
preds = layers.Dense(48, activation="softmax")(x)
 
model = keras.Model(int_seq, preds)

In [11]:
# Read training examples for train and validation from example files,
# Lists xtrain. y train: x and d of training examples from training data.
# Lists X validation. v validation: the same as the above.
 
 
def read_examples_from_file(path_examples_file):
    fp = open(path_examples_file, "r", encoding="utf-8")
    
    X = []
    Y = []
    for i, line in enumerate(fp.readlines()): 
        if i == 0:
            continue # ignore the first line which is a comment
        line_split = line.split()
        intseq = [int(a) for a in line_split]
        an_inputX = intseq[0:wirldow_size]
        X.append(an_inputX) # X (an input word index sea)
        Y.append(intseq[-1]) # d (a label not a 1-hot vector)
    return X, Y
 
x_train, y_train = read_examples_from_file("./all_examples_train.txt")
x_train = np.array(x_train)
y_train = np.array(y_train)
 
 
x_validation, y_validation = read_examples_from_file("./all_examples_validation.txt")
x_validation = np.array(x_validation)
y_validation = np.array(y_validation)
 
x_test, y_test = read_examples_from_file("./all_examples_test.txt")
x_test = np.array(x_test)
y_test = np.array(y_test)

In [12]:
# loss sparse categorical crossentropy: target is given as a label index than one-hot vector.
LEARNING_RATE = 0.3e-3
DECAY = 0.00005
EPOCHS = 5
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, decay=DECAY)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["acc"])
 
model.fit(x_train, y_train, batch_size=64, epochs=EPOCHS, validation_data=(x_validation,
                        y_validation), shuffle=True)

Epoch 1/5


ValueError: ignored

In [ ]:
## TEST ##
pred = model.predict(x=x_test, verbose=1) # batch 단위로 수행함 (default - 32)
                                          # batch 별 결과를 므아서 전체 결과를 반환함
pred_label = tf.math.argmax(pred, axis=1)
leng_test = len(ytest)
equality = tf.math.equal(pred_label, y_test)  # this is a boolean array.
compare = np.ones(leng_test, dtype=np.int64)
for i in range(leng_test):
    if not equality[i]:
        compare[1] = 0
hit = tf.reduce_sum(compare)
acc = hit / leng test
print("test accuracy =", acc)
